In [41]:
pip install cfbd

Note: you may need to restart the kernel to use updated packages.


In [42]:
import os
cfbd_key = os.environ['cfb']

In [43]:
import cfbd
from cfbd.rest import ApiException
import pandas as pd

In [44]:
# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = cfbd_key
configuration.api_key_prefix['Authorization'] = 'Bearer'

In [45]:
# create an instance of the API class
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year = 2022 # int | Year/season filter for games (optional)
# week = 1 # int | Week filter (optional)

try:
    # Betting lines
    api_response = api_instance.get_games(year=year)
    # print(api_response)
except ApiException as e:
    print("Exception when calling BettingApi->get_lines: %s\n" % e)

In [46]:
n_games = len(api_response)

for i in range(0, n_games):
    
    resd = api_response[i].to_dict()
    
    b1 = api_response[i].away_points
    b2 = bool(api_response[i].home_division =='fbs' or api_response[i].away_division =='fbs')
    
    if i == 0: # build the df on the first game
        df = pd.DataFrame.from_dict(resd)
    elif b1 and b2: # skip canceled games and non fbs games
        df = df.append(pd.DataFrame.from_dict(resd))
    # print(i, "of", n_games)

In [47]:
df.drop(['season', 'season_type', 'start_date', 'start_time_tbd',
       'conference_game', 'attendance', 'venue_id',
       'home_id', 'home_conference', 'home_division',
       'home_post_win_prob', 'home_line_scores',
       'home_pregame_elo', 'home_postgame_elo', 'away_id',
       'away_conference', 'away_division', 'away_line_scores',
       'away_post_win_prob', 'away_pregame_elo', 'away_postgame_elo',
       'excitement_index', 'highlights', 'notes'], axis=1, inplace=True)

df.drop_duplicates(inplace=True)

df.set_index('id', inplace=True)

In [48]:
df

,week,neutral_site,venue,home_team,home_points,away_team,away_points
id,,,,,,,
401426532,1,False,Houchens Industries-L. T. Smith Stadium,Western Kentucky,38,Austin Peay,27
401405059,1,True,Aviva Stadium,Northwestern,31,Nebraska,28
401413256,1,False,Allegiant Stadium,UNLV,52,Idaho State,21
401405058,1,False,Memorial Stadium,Illinois,38,Wyoming,6
401404146,1,False,Romney Stadium,Utah State,31,Connecticut,20
...,...,...,...,...,...,...,...
401403973,1,False,Husky Stadium,Washington,45,Kent State,20
401403969,1,False,Reser Stadium,Oregon State,34,Boise State,17
401426543,1,False,Clarence T.C. Ching Athletics Complex,Hawai'i,17,Western Kentucky,49


In [49]:
id_lst = []
ot_lst = []

for i in range(0, n_games):
    
    b1 = api_response[i].away_points
    b2 = bool(api_response[i].home_division =='fbs' or api_response[i].away_division =='fbs')
    
    if b1 and b2:
        _ot = bool(len(api_response[i].home_line_scores) > 4)
        ot_lst.append(_ot)
        _id = api_response[i].id
        id_lst.append(_id)

In [51]:
df_ot = pd.DataFrame()
df_ot['id'] = id_lst
df_ot['ot'] = ot_lst
df_ot.set_index('id', inplace=True)

df = df.join(df_ot)

In [52]:
df[(df['ot']==True)]

,week,neutral_site,venue,home_team,home_points,away_team,away_points,ot
id,,,,,,,,
401416568,1,False,Summa Field at InfoCision Stadium,Akron,30,St Francis (PA),23,True
401426533,1,False,FIU Stadium,Florida International,38,Bryant,37,True
401415211,1,False,War Memorial Stadium,Wyoming,40,Tulsa,37,True
401426542,1,False,Alamodome,UT San Antonio,35,Houston,37,True
401404157,1,False,M. M. Roberts Stadium,Southern Mississippi,27,Liberty,29,True
